In [15]:
import cv2
import numpy as np


In [16]:
model_weights = "yolov3-tiny-obj_5000.weights"
model_config = "yolov3-tiny-obj.cfg"

In [17]:
# Load tiny Yolo custom trained weights
net = cv2.dnn.readNetFromDarknet(model_config, model_weights)

classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
#colors = np.random.uniform(0, 255, size=(len(classes), 3))
colors = (255,255,255)

In [18]:
"""
# Loading image
img = cv2.imread("rocket+man.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape
"""

'\n# Loading image\nimg = cv2.imread("rocket+man.jpg")\nimg = cv2.resize(img, None, fx=0.4, fy=0.4)\nheight, width, channels = img.shape\n'

In [23]:
print("Enter 1 for webcam.\n /anything else for video file mode")
input_choice = int(input())
if input_choice == 1:
    print("Starting webcam")
    cap = cv2.VideoCapture(0)
else:
    #print("Enter video file name/path")
    video_file = "launch-video-1.mp4"
    cap = cv2.VideoCapture(video_file)

font = cv2.FONT_HERSHEY_PLAIN

while(True):
    # Capture frame-by-frame
    ret, img = cap.read()
    
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    #img = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA)
    height, width, channels = img.shape
    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3: #we will lower this since we only have 1 class.
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) #we’ll apply non-maxima suppression
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i-1]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y), font, 1, color, 1)
    
    
    # Display the resulting frame
    cv2.namedWindow('frame', cv2.WINDOW_FREERATIO)
    #cv2.cvSetWindowProperty('frame', cv2.CV_WND_PROP_FULLSCREEN, cv2.CV_WINDOW_FULLSCREEN);
    cv2.imshow('frame',img);

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Enter 1 for webcam.
 /anything else for video file mode
2


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4045: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [20]:
"""

# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)
"""

'\n\n# Detecting objects\nblob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)\nnet.setInput(blob)\nouts = net.forward(output_layers)\n'

In [21]:
"""
# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
"""

'\n# Showing informations on the screen\nclass_ids = []\nconfidences = []\nboxes = []\nfor out in outs:\n    for detection in out:\n        scores = detection[5:]\n        class_id = np.argmax(scores)\n        confidence = scores[class_id]\n        if confidence > 0.5:\n            # Object detected\n            center_x = int(detection[0] * width)\n            center_y = int(detection[1] * height)\n            w = int(detection[2] * width)\n            h = int(detection[3] * height)\n            # Rectangle coordinates\n            x = int(center_x - w / 2)\n            y = int(center_y - h / 2)\n            boxes.append([x, y, w, h])\n            confidences.append(float(confidence))\n            class_ids.append(class_id)\n'

In [22]:
"""
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) #we’ll apply non-maxima suppression
"""

'\nindexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) #we’ll apply non-maxima suppression\n'

In [9]:

"""
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

'\nfont = cv2.FONT_HERSHEY_PLAIN\nfor i in range(len(boxes)):\n    if i in indexes:\n        x, y, w, h = boxes[i]\n        label = str(classes[class_ids[i]])\n        color = colors[i]\n        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)\n        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)\ncv2.imshow("Image", img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'